# Clone data and dependencies

In [1]:
!pip install underthesea
!pip install nltk
!pip install transformers
!pip install cython
!pip install youtokentome
!pip install datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [3]:
%cd /gdrive/My\ Drive/Group\ 3 - NLP\ Project/source_code

/gdrive/My Drive/Group 3 - NLP Project/source_code


# Import our model's modules

In [4]:
from src.vocabulary import Vocabulary
from src.dataset import StanfordEnViDataset,get_loader
from src.model import EnViTransformer
from src.utils import save_checkpoint, load_checkpoint, training_step, validate, beam_search
from src.Translator import TransformerTranslator

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Next, we will load all of our trained model, and demo them with some sentences

## Firstly, models trained with different input types

Model trained on word segmentation

In [ ]:
import yaml
from yaml.loader import SafeLoader
import torch
import datetime
with open('./utils/config/word_segmentation_config.yml') as f:
    segment_config = yaml.load(f, Loader=SafeLoader)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

segmentTranslator = TransformerTranslator(segment_config, device, input_type = 'word_segmentation')
print(len(segmentTranslator.vi_vocab), len(segmentTranslator.en_vocab))


=> Loading checkpoint
16250 22407


In [ ]:
sentences = ['She went to Stanford University',
            "I'm very proud to present my work",
             "Let me tell you about the fifth-grade students at Boze Elementary School in Tacoma. They spend the entire school year planning to colonize the planet Alpha Centauri to preserve future generations. They will get there on a multigenerational spacecraft so that the grandkids of the people on the ship will arrive at Alpha Centauri, ready to start a new civilization."]
segmentTranslator.inference(sentences)

=> Loading checkpoint


[' Bà đến trường  Đại học  Stanford ',
 ' Tôi rất tự hào về việc hiện tại công việc của tôi ',
 ' Tôi sẽ kể cho các bạn nghe về những học sinh trung học ở trường  Tiểu học ở  <UNK>  . họ dành toàn bộ năm chuẩn bị lập kế hoạch cho các thế hệ  <UNK> để bảo tồn thế hệ mới .  Họ sẽ đến đó trên một tàu vũ trụ để đối diện với các con tàu trên tàu  <UNK> , đã đưa ra một nền giáo dục mới. ']

Model trained on simple tokenization

In [ ]:
with open('./utils/config/tokenization_config.yml') as f:
  tokenize_config = yaml.load(f, Loader=SafeLoader)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizeTranslator = TransformerTranslator(tokenize_config, device, input_type = 'tokenization')
print(len(tokenizeTranslator.vi_vocab), len(tokenizeTranslator.en_vocab))


=> Loading checkpoint
9033 22407


In [ ]:
sentences = ['She went to Stanford University',
            "I'm very proud to present my work",
             "Let me tell you about the fifth-grade students at Boze Elementary School in Tacoma. They spend the entire school year planning to colonize the planet Alpha Centauri to preserve future generations. They will get there on a multigenerational spacecraft so that the grandkids of the people on the ship will arrive at Alpha Centauri, ready to start a new civilization."]
tokenizeTranslator.inference(sentences)

=> Loading checkpoint


[' Cô ấy đến trường đại học  Đại học  Stanford. ',
 ' Tôi rất tự hào để hiện hữu công việc của mình. ',
 ' Để tôi nói với các bạn về sinh viên ở  Ngôi trường  <UNK>  <UNK> ở  <UNK> .  Họ tiêu hết toàn bộ năm học để đánh giá hành tinh  <UNK> để gìn giữ lại hệ thống tương lai .  Họ sẽ ở đó một chuyến tàu vũ trụ mà con tàu con người trên tàu sẽ đến với  Alpha  <UNK> , sẵn sàng để khởi đầu một nền văn minh mới. ']

Model trained on bpe

In [ ]:
with open('./utils/config/bpe_config.yml') as f:
    bpe_config = yaml.load(f, Loader=SafeLoader)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

bpeTranslator = TransformerTranslator(bpe_config, device, input_type = 'bpe')
print(len(bpeTranslator.vi_vocab), len(bpeTranslator.en_vocab))


=> Loading checkpoint
4231 4622


In [ ]:
sentences = ['She went to Stanford University',
            "I'm very proud to present my work",
             "Let me tell you about the fifth-grade students at Boze Elementary School in Tacoma. They spend the entire school year planning to colonize the planet Alpha Centauri to preserve future generations. They will get there on a multigenerational spacecraft so that the grandkids of the people on the ship will arrive at Alpha Centauri, ready to start a new civilization."]
bpeTranslator.inference(sentences)

=> Loading checkpoint


[' Cô đến trường Đại học Đại học Học Stanford ',
 ' Tôi rất tự hào khi trình diễn công việc của mình. ',
 ' Hãy để tôi kể cho các bạn về những sinh viên năm tháng 5 năm học tại Bozeree Đại học Học ở Tacomima Họ đã dành toàn bộ năm kế hoạch hoá học tập trung toàn bộ hành trình Alphera Trung Tâm Quốc để bảo tồn các thế hệ tương lai . Họ sẽ nhận được một vũ trụ ở đó trên tàu vũ trang Vũ trụ sẽ cho phép người ta ở New York Trí ả Phi. ']

## Next, models trained on augmented data

Model trained with synonym replacement

In [ ]:
import yaml
from yaml.loader import SafeLoader
import torch
import datetime
with open('./utils/config/synonym_replacement_config.yml') as f:
  synonym_replacement_config = yaml.load(f, Loader=SafeLoader)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

synonym_replacement_Translator = TransformerTranslator(synonym_replacement_config, device, input_type = 'word_segmentation')
print(len(synonym_replacement_Translator.vi_vocab), len(synonym_replacement_Translator.en_vocab))


=> Loading checkpoint
20113 29631


In [ ]:
sentences = ['She went to Stanford University', "Let me tell you about the fifth-grade students at Boze Elementary School in Tacoma. They spend the entire school year planning to colonize the planet Alpha Centauri to preserve future generations. They will get there on a multigenerational spacecraft so that the grandkids of the people on the ship will arrive at Alpha Centauri, ready to start a new civilization."]
synonym_replacement_Translator.inference(sentences)

=> Loading checkpoint


[' Cô ấy đến trường đại học  Đại học  Stanford ',
 ' Để tôi kể cho các bạn nghe về những học sinh ở  <UNK> ở  <UNK> , ở  <UNK> , ở  <UNK> , họ dành cả năm để nâng lên kế hoạch của trái đất  <UNK> để bảo tồn các thế hệ tương lai  . trong tương lai  Họ sẽ có trên một tàu vũ trụ để mọi người trên chuyến tàu trên tàu sẽ đến  <UNK> , sẵn sàng để bắt đầu một nền văn minh mới. ']

Model trained with EN-FR translation

In [ ]:
import yaml
from yaml.loader import SafeLoader
import torch
import datetime
with open('./utils/config/en_fr_config.yml') as f:
    en_fr_config = yaml.load(f, Loader=SafeLoader)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

en_fr_Translator = TransformerTranslator(en_fr_config, device, input_type = 'word_segmentation')
print(len(en_fr_Translator.vi_vocab), len(en_fr_Translator.en_vocab))


=> Loading checkpoint
20100 26685


In [ ]:
sentences = ['She went to Stanford University', "Let me tell you about the fifth-grade students at Boze Elementary School in Tacoma. They spend the entire school year planning to colonize the planet Alpha Centauri to preserve future generations. They will get there on a multigenerational spacecraft so that the grandkids of the people on the ship will arrive at Alpha Centauri, ready to start a new civilization."]
en_fr_Translator.inference(sentences)

=> Loading checkpoint


[' Cô ấy đến trường  Đại học  Stanford ',
 ' Hãy để tôi nói cho các bạn biết về những học sinh lớp 4 ở trường  <UNK> ở  <UNK> .  Họ dùng cả trường để lập bản đồ của hành tinh  <UNK> để gìn giữ các thế hệ tương lai  . họ sẽ đạt được một tàu thám hiểm lớn để những người dân trên tàu sẽ đến  Đại  Tây dương , sẵn sàng để lập một nền văn minh mới. ']

Model trained with contractions

In [ ]:
import yaml
from yaml.loader import SafeLoader
import torch
import datetime
with open('./utils/config/contractions_config.yml') as f:
    contractions_config = yaml.load(f, Loader=SafeLoader)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

contractions_Translator = TransformerTranslator(contractions_config, device, input_type = 'word_segmentation')
print(len(contractions_Translator.vi_vocab), len(contractions_Translator.en_vocab))


=> Loading checkpoint
19438 26641


In [ ]:
sentences = ['She went to Stanford University', "Let me tell you about the fifth-grade students at Boze Elementary School in Tacoma. They spend the entire school year planning to colonize the planet Alpha Centauri to preserve future generations. They will get there on a multigenerational spacecraft so that the grandkids of the people on the ship will arrive at Alpha Centauri, ready to start a new civilization."]
contractions_Translator.inference(sentences)

=> Loading checkpoint


[' Cô ấy đến trường đại học  Stanford. ',
 ' Để tôi nói với các bạn về những sinh viên ở trường tiểu học  <UNK> ở  Đại học  <UNK> .  Họ dành cả năm để trường đào tạo cho việc huỷ hoại hành tinh này  <UNK> để gìn giữ những thế hệ tương lai  . chúng sẽ có trên một tàu vũ trụ dành cho các con tàu vũ trụ sẽ đến trường  <UNK> , sẵn sàng bắt đầu một nền văn minh của  <UNK>. ']

Model trained with back translation

In [ ]:
import yaml
from yaml.loader import SafeLoader
import torch
import datetime
with open('./utils/config/back_translation_config.yml') as f:
  back_translation_config = yaml.load(f, Loader=SafeLoader)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

back_translation_Translator = TransformerTranslator(back_translation_config, device, input_type = 'word_segmentation')
print(len(back_translation_Translator.vi_vocab), len(back_translation_Translator.en_vocab))


=> Loading checkpoint
21703 27613


In [ ]:
sentences = ['She went to Stanford University', "Let me tell you about the fifth-grade students at Boze Elementary School in Tacoma. They spend the entire school year planning to colonize the planet Alpha Centauri to preserve future generations. They will get there on a multigenerational spacecraft so that the grandkids of the people on the ship will arrive at Alpha Centauri, ready to start a new civilization."]
back_translation_Translator.inference(sentences)

=> Loading checkpoint


[' Cô ấy đến trường đại học  Stanford ',
 ' Hãy để tôi nói với các bạn về những sinh viên ở trường tiểu học  <UNK> ở  <UNK>  <UNK> ,  Họ dành toàn bộ năm lên kế hoạch cho  <UNK> để xâm chiếm hành tinh  <UNK> để bảo tồn những thế hệ tương lai  . họ sẽ đến đó trên một con tàu vũ trụ <UNK> để cho những người dân trên con tàu sẽ đến  <UNK> , sẵn sàng để bắt đầu một nền văn minh mới. ']

# Next, we run our models on the test set, save the results and evaluate BLEU scores

Read the files

In [ ]:
# import underthesea
en_test_sents = []
with open("./data/processed/test/tst2013.en") as f:
    for sent in f:
        sent  = sent.strip()
        en_test_sents.append(sent)

vi_test_sents = []
with open("./data/processed/test/tst2013.vi") as f:
    for sent in f:
        sent  = sent.strip()
        # sent =  underthesea.word_tokenize(sent)
        # if len(sent) > 80:
        #   sent = sent[0:80]
        # sent  = ((" ".join(sent))).strip()
        vi_test_sents.append(sent)


In [ ]:
def save_texts(texts,path):
  with open(path, 'w') as f:
    for line in texts:
      f.write(line + '\n')

# save_texts(synonym_replacement_translations, './data/translations/synonym_replacement.txt')

Model trained with BPE as input

In [ ]:
bpeTranslations = bpeTranslator.inference(en_test_sents)
save_texts(bpeTranslations, './data/translations/bpe.txt')

=> Loading checkpoint


Model trained with simple tokenization as input

In [ ]:
tokenizeTranslations = tokenizeTranslator.inference(en_test_sents)
save_texts(tokenizeTranslations, './data/translations/tokenize.txt')

=> Loading checkpoint


Model trained with word segmentation as input

In [ ]:
segmentTranslations = segmentTranslator.inference(en_test_sents)
save_texts(segmentTranslations, './data/translations/segmentation.txt')

=> Loading checkpoint


Model trained with synonym replacements

In [ ]:
synonym_replacement_translations = synonym_replacement_Translator.inference(en_test_sents)
save_texts(synonym_replacement_translations, './data/translations/synonym_replacement.txt')

=> Loading checkpoint


Model trained with back translations

In [ ]:
backtr_translations = back_translation_Translator.inference(en_test_sents)
save_texts(backtr_translations, './data/translations/back_translations.txt')

=> Loading checkpoint


Model trained with contractions

In [ ]:
contractions_translations = contractions_Translator.inference(en_test_sents)
save_texts(contractions_translations, './data/translations/contractions.txt')

=> Loading checkpoint


Model trained with English-French translations

In [ ]:
en_fr_Translations = en_fr_Translator.inference(en_test_sents)
save_texts(en_fr_Translations, './data/translations/english-french.txt')

=> Loading checkpoint


Save the results in text files

In [ ]:
with open('./data/translations/bpe.txt') as f:
  bpe_translations = [line.strip() for line in f.readlines()]

with open('./data/translations/tokenize.txt') as f:
  tokenize_translations = [line.strip() for line in f.readlines()]

with open('./data/translations/segmentation.txt') as f:
  segment_translations = [line.strip() for line in f.readlines()]

with open('./data/translations/back_translations.txt') as f:
  backtr_translations = [line.strip() for line in f.readlines()]

with open('./data/translations/synonym_replacement.txt') as f:
  synonym_replacement_translations = [line.strip() for line in f.readlines()]

with open('./data/translations/contractions.txt') as f:
  contractions_translations = [line.strip() for line in f.readlines()]

with open('./data/translations/english-french.txt') as f:
  en_fr_translations = [line.strip() for line in f.readlines()]


# Evaluate the BLEU score

In [ ]:
! pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sacrebleu.metrics import BLEU
def calc_bleu(targets, translations, case_sensitive = False):
  if case_sensitive:
    print("Calculating cased BLEU")
    targets = [sentence.strip() for sentence in targets]
    sys = [translation.strip() for translation in translations] 
  else:
    print("Calculating uncased BLEU")
    targets = [sentence.strip().lower() for sentence in targets]
    sys = [translation.strip().lower() for translation in translations] 

  refs = [targets]
  bleu = BLEU()
  a = bleu.corpus_score(sys, refs)
  print(a)

Model trained with BPE as input

In [ ]:
calc_bleu(vi_test_sents, bpe_translations, case_sensitive = False)
calc_bleu(vi_test_sents, bpeTranslations, case_sensitive = True)

Calculating uncased BLEU
BLEU = 25.56 58.7/32.9/19.4/11.8 (BP = 0.992 ratio = 0.992 hyp_len = 33444 ref_len = 33727)
Calculating cased BLEU
BLEU = 24.79 57.5/31.9/18.8/11.3 (BP = 0.992 ratio = 0.992 hyp_len = 33444 ref_len = 33727)


Model trained with simple tokenization as input

In [ ]:
calc_bleu(vi_test_sents, tokenize_translations, case_sensitive = False)
calc_bleu(vi_test_sents, tokenize_translations, case_sensitive = True)

Calculating uncased BLEU
BLEU = 25.19 58.0/32.2/18.9/11.4 (BP = 1.000 ratio = 1.002 hyp_len = 33789 ref_len = 33727)
Calculating cased BLEU
BLEU = 24.37 56.8/31.3/18.2/10.9 (BP = 1.000 ratio = 1.002 hyp_len = 33789 ref_len = 33727)


Model trained with word segmentation as input

In [ ]:
calc_bleu(vi_test_sents, segment_translations, case_sensitive = False)
calc_bleu(vi_test_sents, segment_translations, case_sensitive = True)

Calculating uncased BLEU
BLEU = 25.83 59.2/33.5/19.9/12.1 (BP = 0.983 ratio = 0.983 hyp_len = 33143 ref_len = 33727)
Calculating cased BLEU
BLEU = 24.69 57.7/32.2/18.9/11.3 (BP = 0.983 ratio = 0.983 hyp_len = 33143 ref_len = 33727)


Model trained with synonym replacement

In [ ]:
calc_bleu(vi_test_sents, synonym_replacement_translations, case_sensitive = False)
calc_bleu(vi_test_sents,synonym_replacement_translations, case_sensitive = True)

Calculating uncased BLEU
BLEU = 27.56 60.4/34.9/21.2/13.0 (BP = 0.998 ratio = 0.998 hyp_len = 33671 ref_len = 33727)
Calculating cased BLEU
BLEU = 26.39 59.0/33.6/20.1/12.3 (BP = 0.998 ratio = 0.998 hyp_len = 33671 ref_len = 33727)


Model trained with contractions

In [ ]:
calc_bleu(vi_test_sents, contractions_translations, case_sensitive = False)
calc_bleu(vi_test_sents,contractions_translations, case_sensitive = True)

Calculating uncased BLEU
BLEU = 26.84 59.8/34.2/20.5/12.7 (BP = 0.994 ratio = 0.994 hyp_len = 33525 ref_len = 33727)
Calculating cased BLEU
BLEU = 25.71 58.4/32.9/19.5/11.9 (BP = 0.994 ratio = 0.994 hyp_len = 33525 ref_len = 33727)


Model trained with back translation

In [ ]:
calc_bleu(vi_test_sents, backtr_translations, case_sensitive = False)
calc_bleu(vi_test_sents,backtr_translations, case_sensitive = True)

Calculating uncased BLEU
BLEU = 29.01 61.8/36.5/22.4/14.0 (BP = 1.000 ratio = 1.004 hyp_len = 33859 ref_len = 33727)
Calculating cased BLEU
BLEU = 27.69 60.2/35.0/21.2/13.1 (BP = 1.000 ratio = 1.004 hyp_len = 33859 ref_len = 33727)


Model trained with EN-FR translation

In [ ]:
calc_bleu(vi_test_sents, en_fr_Translations, case_sensitive = False)
calc_bleu(vi_test_sents, en_fr_Translations, case_sensitive = True)

Calculating uncased BLEU
BLEU = 27.53 60.0/34.6/21.1/13.1 (BP = 1.000 ratio = 1.015 hyp_len = 34229 ref_len = 33727)
Calculating cased BLEU
BLEU = 26.34 58.5/33.3/20.1/12.3 (BP = 1.000 ratio = 1.015 hyp_len = 34229 ref_len = 33727)
